## Import libraries 

In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
model_path = "./1544253016"

## Define feature columns

In [ ]:
_HASH_BUCKET_SIZE = 1000

# Continuous variable columns
age = tf.feature_column.numeric_column('age')
education_num = tf.feature_column.numeric_column('education-num')
capital_gain = tf.feature_column.numeric_column('capital-gain')
capital_loss = tf.feature_column.numeric_column('capital-loss')
hours_per_week = tf.feature_column.numeric_column('hours-per-week')
gender = tf.feature_column.categorical_column_with_vocabulary_list(
      'sex', ['Male', 'Female'])
education = tf.feature_column.categorical_column_with_vocabulary_list(
      'education', ['Bachelors', 'HS-grad', '11th', 'Masters', '9th', 'Some-college',
          'Assoc-acdm', 'Assoc-voc', '7th-8th', 'Doctorate', 'Prof-school',
          '5th-6th', '10th', '1st-4th', 'Preschool', '12th'])

marital_status = tf.feature_column.categorical_column_with_vocabulary_list(
      'marital-status', ['Married-civ-spouse', 'Divorced', 'Married-spouse-absent',
          'Never-married', 'Separated', 'Married-AF-spouse', 'Widowed'])

relationship = tf.feature_column.categorical_column_with_vocabulary_list(
      'relationship', ['Husband', 'Not-in-family', 'Wife', 'Own-child', 'Unmarried',
          'Other-relative'])

workclass = tf.feature_column.categorical_column_with_vocabulary_list(
      'workclass', ['Self-emp-not-inc', 'Private', 'State-gov', 'Federal-gov',
          'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked'])

# To show an example of hashing:
occupation = tf.feature_column.categorical_column_with_hash_bucket(
      'occupation', hash_bucket_size=_HASH_BUCKET_SIZE)

# Transformations.
age_buckets = tf.feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])

# Wide columns and deep columns.
base_columns = [education, marital_status, relationship, workclass, occupation,age_buckets,gender,]

crossed_columns = [
      tf.feature_column.crossed_column(['education', 'occupation'], hash_bucket_size=_HASH_BUCKET_SIZE),
      tf.feature_column.crossed_column([age_buckets, 'education', 'occupation'],hash_bucket_size=_HASH_BUCKET_SIZE),]

wide_columns = base_columns + crossed_columns

deep_columns = [
      age,
      education_num,
      capital_gain,
      capital_loss,
      hours_per_week,
      tf.feature_column.indicator_column(gender),
      tf.feature_column.indicator_column(workclass),
      tf.feature_column.indicator_column(education),
      tf.feature_column.indicator_column(marital_status),
      tf.feature_column.indicator_column(relationship),
      # To show an example of embedding
      tf.feature_column.embedding_column(occupation, dimension=8),
  ]


In [ ]:
hidden_units = [100, 75, 50, 25]

## Use DNN classifier

In [ ]:
classifier = tf.estimator.DNNClassifier(
        feature_columns=deep_columns,
        hidden_units=hidden_units,
        n_classes=2)

## Use Deep & Wide model

In [ ]:
classifier = tf.estimator.DNNLinearCombinedClassifier(
        linear_feature_columns=wide_columns,
        dnn_feature_columns=deep_columns,
        dnn_hidden_units=hidden_units,
        warm_start_from=model_path)

## Setup training & evaluation pipeline

In [ ]:
NUM_EPOCHS = 30
BUFFER_SIZE= 32561

def parse_func(s_example):
    features = {
                'workclass': tf.FixedLenFeature((), tf.string),
                'age': tf.FixedLenFeature((), tf.int64),
                'fnlwgt': tf.FixedLenFeature((), tf.int64),
                'education': tf.FixedLenFeature((), tf.string),
                'education-num': tf.FixedLenFeature((), tf.int64),
                'marital-status': tf.FixedLenFeature((), tf.string),
                'occupation': tf.FixedLenFeature((), tf.string),
                'relationship': tf.FixedLenFeature((), tf.string),
                'race': tf.FixedLenFeature((), tf.string),
                'sex': tf.FixedLenFeature((), tf.string),
                'capital-gain': tf.FixedLenFeature((), tf.int64),
                'capital-loss': tf.FixedLenFeature((), tf.int64),
                'hours-per-week': tf.FixedLenFeature((), tf.int64),
                'native-country': tf.FixedLenFeature((), tf.string),
                'label': tf.FixedLenFeature((), tf.string)
                 }
    example = tf.parse_single_example(s_example, features=features)
    return example

def transform(features):
    for key in ['workclass','education','marital-status','occupation','relationship','race','sex','native-country','label']:
        features[key] = tf.strings.strip(features[key])
    labels= features.pop('label')
    return features,tf.cast(tf.logical_or(tf.equal(labels,'>50K'),tf.equal(labels,'>50K.')),tf.int32)


def eval_input_fn(batch_size):
    return tf.data.TFRecordDataset('census_test.tfrecord').map(parse_func).map(transform).batch(batch_size)
def train_input_fn(batch_size):
    return tf.data.TFRecordDataset('census.tfrecord').shuffle(BUFFER_SIZE).repeat(NUM_EPOCHS).map(parse_func).map(transform).batch(batch_size)



## Train the model for 30 epochs

In [ ]:
r = classifier.train(input_fn=lambda:train_input_fn(256))

## Evaluate the model 

In [ ]:
eval_result = classifier.evaluate(input_fn=lambda:eval_input_fn(256))
eval_result

## Save the model

In [ ]:
feature_spec = {
                'workclass': tf.FixedLenFeature((), tf.string),
                'age': tf.FixedLenFeature((), tf.int64),
                'education': tf.FixedLenFeature((), tf.string),
                'education-num': tf.FixedLenFeature((), tf.int64),
                'marital-status': tf.FixedLenFeature((), tf.string),
                'occupation': tf.FixedLenFeature((), tf.string),
                'relationship': tf.FixedLenFeature((), tf.string),
                'sex': tf.FixedLenFeature((), tf.string),
                'capital-gain': tf.FixedLenFeature((), tf.int64),
                'capital-loss': tf.FixedLenFeature((), tf.int64),
                'hours-per-week': tf.FixedLenFeature((), tf.int64),
                 }
"""
my_feature_columns = [age,education_num,capital_gain,capital_loss,occupation,workclass,relationship,marital_status,
                  education,hours_per_week]
feature_spec= tf.feature_column.make_parse_example_spec(my_feature_columns)
"""
export_input_fn= tf.estimator.export.build_parsing_serving_input_receiver_fn(feature_spec)
export_dir = classifier.export_savedmodel(export_dir_base=".",serving_input_receiver_fn=export_input_fn)

### Show the path to the saved model

In [ ]:
export_dir